### The ugly truth behind beauty: A deep dive into chemicals in cosmetics.

In [166]:
import pandas as pd

### Load the data

In [167]:
df = pd.read_csv("cscpopendata.csv", low_memory = False)
df.head()

,CDPHId,ProductName,CSFId,CSF,CompanyId,CompanyName,BrandName,PrimaryCategoryId,PrimaryCategory,SubCategoryId,...,CasNumber,ChemicalId,ChemicalName,InitialDateReported,MostRecentDateReported,DiscontinuedDate,ChemicalCreatedAt,ChemicalUpdatedAt,ChemicalDateRemoved,ChemicalCount
0,2,ULTRA COLOR RICH EXTRA PLUMP LIPSTICK-ALL SHADES,NaN,NaN,4,New Avon LLC,AVON,44,Makeup Products (non-permanent),53,...,13463-67-7,6,Titanium dioxide,06/17/2009,08/28/2013,02/01/2011,07/09/2009,07/09/2009,NaN,1
1,3,Glover's Medicated Shampoo,NaN,NaN,338,J. Strickland & Co.,Glover's,18,Hair Care Products (non-coloring),25,...,65996-92-1,4,Distillates (coal tar),07/01/2009,07/01/2009,NaN,07/01/2009,07/01/2009,NaN,2
2,3,Glover's Medicated Shampoo,NaN,NaN,338,J. Strickland & Co.,Glover's,18,Hair Care Products (non-coloring),25,...,140-67-0,5,Estragole,07/01/2009,07/01/2009,NaN,07/02/2009,07/02/2009,NaN,2
3,4,PRECISION GLIMMER EYE LINER-ALL SHADES �,NaN,NaN,4,New Avon LLC,AVON,44,Makeup Products (non-permanent),46,...,13463-67-7,7,Titanium dioxide,07/09/2009,08/28/2013,NaN,07/09/2009,07/09/2009,NaN,1
4,5,AVON BRILLIANT SHINE LIP GLOSS-ALL SHADES �,NaN,NaN,4,New Avon LLC,AVON,44,Makeup Products (non-permanent),52,...,13463-67-7,8,Titanium dioxide,07/09/2009,08/28/2013,02/01/2011,07/09/2009,07/09/2009,NaN,1


In [172]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114635 entries, 0 to 114634
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   CDPHId                  114635 non-null  int64  
 1   ProductName             114635 non-null  object 
 2   CSFId                   80662 non-null   float64
 3   CSF                     80237 non-null   object 
 4   CompanyId               114635 non-null  int64  
 5   CompanyName             114635 non-null  object 
 6   BrandName               114408 non-null  object 
 7   PrimaryCategoryId       114635 non-null  int64  
 8   PrimaryCategory         114635 non-null  object 
 9   SubCategoryId           114635 non-null  int64  
 10  SubCategory             114635 non-null  object 
 11  CasId                   114635 non-null  int64  
 12  CasNumber               108159 non-null  object 
 13  ChemicalId              114635 non-null  int64  
 14  ChemicalName        

In [170]:
print(df.isnull().sum())

CDPHId                         0
ProductName                    0
CSFId                      33973
CSF                        34398
CompanyId                      0
CompanyName                    0
BrandName                    227
PrimaryCategoryId              0
PrimaryCategory                0
SubCategoryId                  0
SubCategory                    0
CasId                          0
CasNumber                   6476
ChemicalId                     0
ChemicalName                   0
InitialDateReported            0
MostRecentDateReported         0
DiscontinuedDate          101715
ChemicalCreatedAt              0
ChemicalUpdatedAt              0
ChemicalDateRemoved       111650
ChemicalCount                  0
dtype: int64


Percentage of missing values.

In [174]:
missing_values = df.isnull().sum() / len(df) * 100
missing_values[missing_values > 0]

CSFId                  29.635801
CSF                    30.006543
BrandName               0.198020
CasNumber               5.649235
DiscontinuedDate       88.729446
ChemicalDateRemoved    97.396083
dtype: float64

In [176]:
df.columns

Index(['CDPHId', 'ProductName', 'CSFId', 'CSF', 'CompanyId', 'CompanyName',
       'BrandName', 'PrimaryCategoryId', 'PrimaryCategory', 'SubCategoryId',
       'SubCategory', 'CasId', 'CasNumber', 'ChemicalId', 'ChemicalName',
       'InitialDateReported', 'MostRecentDateReported', 'DiscontinuedDate',
       'ChemicalCreatedAt', 'ChemicalUpdatedAt', 'ChemicalDateRemoved',
       'ChemicalCount'],
      dtype='object')

In [178]:
#CasNumber = REAL global chemical ID (like a passport number for a chemical).

df = df.drop(columns=["DiscontinuedDate", "ChemicalDateRemoved", "CasNumber"])


When these unique identifiers like CasId, CDPHId etc were removed, previously unique rows became identical, increasing the number of duplicate rows. so lets only handle columns that have null values. 

If dropping a column introduces duplicates, that means it's playing a role in differentiating data.
Keeping them (even if unused in analysis) prevents accidental data distortion.


In [181]:
df["CSFId"].fillna(0, inplace=True)  # Filling missing CSFId with 0
df["CSF"].fillna("Unknown", inplace=True)  # Filling missing CSF with "Unknown"


Since the missing values of BrandName are small in number, we can replace them with "other". 

In [183]:
df["BrandName"] = df["BrandName"].fillna("OTHER")

In [185]:
# Check for duplicate rows
duplicate_count = df.duplicated().sum()
print("Duplicate rows before removal:", duplicate_count)

# Drop duplicate rows (keeping the first occurrence)
df = df.drop_duplicates()

# Verify removal
print("Duplicate rows after removal:", df.duplicated().sum())


Duplicate rows before removal: 257
Duplicate rows after removal: 0


In [187]:
#converting datatypes so that we can analyse trends over time without any formatting issues.

# List of date columns
date_columns = ["InitialDateReported", "MostRecentDateReported", "ChemicalCreatedAt", "ChemicalUpdatedAt"]

# Convert to datetime format
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors="coerce")

# Verify changes
print(df.dtypes)


CDPHId                             int64
ProductName                       object
CSFId                            float64
CSF                               object
CompanyId                          int64
CompanyName                       object
BrandName                         object
PrimaryCategoryId                  int64
PrimaryCategory                   object
SubCategoryId                      int64
SubCategory                       object
CasId                              int64
ChemicalId                         int64
ChemicalName                      object
InitialDateReported       datetime64[ns]
MostRecentDateReported    datetime64[ns]
ChemicalCreatedAt         datetime64[ns]
ChemicalUpdatedAt         datetime64[ns]
ChemicalCount                      int64
dtype: object


In [189]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 114378 entries, 0 to 114634
Data columns (total 19 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   CDPHId                  114378 non-null  int64         
 1   ProductName             114378 non-null  object        
 2   CSFId                   114378 non-null  float64       
 3   CSF                     114378 non-null  object        
 4   CompanyId               114378 non-null  int64         
 5   CompanyName             114378 non-null  object        
 6   BrandName               114378 non-null  object        
 7   PrimaryCategoryId       114378 non-null  int64         
 8   PrimaryCategory         114378 non-null  object        
 9   SubCategoryId           114378 non-null  int64         
 10  SubCategory             114378 non-null  object        
 11  CasId                   114378 non-null  int64         
 12  ChemicalId              114378 non-

In [191]:
# Text standardize
# Text standardization

# List of categorical columns to standardize
text_columns = ["BrandName", "CompanyName", "ProductName", "PrimaryCategory", "SubCategory", "ChemicalName", "CSF"]

# Convert text to lowercase, strip spaces, and normalize spacing between words
df[text_columns] = df[text_columns].apply(lambda x: x.str.lower().str.strip().str.replace(r"\s+", " ", regex=True))



In [193]:
# for the ease of analysis, shortened the name.
# Rename long chemical names for better visualization
chemical_name_replacements = {
    "retinol/retinyl esters, when in daily dosages in excess of 10,000 iu, or 3,000 retinol equivalents.": "Retinol/Retinyl Esters",
    "silica, crystalline (airborne particles of respirable size)": "Silica, Crystalline"
}

df["ChemicalName"] = df["ChemicalName"].replace(chemical_name_replacements)


In [195]:
# Export cleaned dataset for Tableau
df.to_csv("cleaned_cosmetics_data.csv", index=False)


## EDA

In [199]:
# checking the information of the dataset

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 114378 entries, 0 to 114634
Data columns (total 19 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   CDPHId                  114378 non-null  int64         
 1   ProductName             114378 non-null  object        
 2   CSFId                   114378 non-null  float64       
 3   CSF                     114378 non-null  object        
 4   CompanyId               114378 non-null  int64         
 5   CompanyName             114378 non-null  object        
 6   BrandName               114378 non-null  object        
 7   PrimaryCategoryId       114378 non-null  int64         
 8   PrimaryCategory         114378 non-null  object        
 9   SubCategoryId           114378 non-null  int64         
 10  SubCategory             114378 non-null  object        
 11  CasId                   114378 non-null  int64         
 12  ChemicalId              114378 non-

In [201]:
# summary and overview

print("Dataset shape:", df.shape)


Dataset shape: (114378, 19)


In [203]:
# Summary statistics for numeric columns
print(df.describe())


              CDPHId          CSFId      CompanyId  PrimaryCategoryId  \
count  114378.000000  114378.000000  114378.000000      114378.000000   
mean    20338.284347   22992.995183     451.187204          51.125767   
min         2.000000       0.000000       4.000000           1.000000   
25%      8791.250000       0.000000      86.000000          44.000000   
50%     20911.500000   19113.500000     298.000000          44.000000   
75%     31380.750000   42733.000000     798.000000          59.000000   
max     41524.000000   65009.000000    1391.000000         111.000000   
std     12481.370666   21865.018812     409.717683          20.464540   

       SubCategoryId          CasId     ChemicalId  \
count  114378.000000  114378.000000  114378.000000   
mean       66.886097     673.139380   32910.498549   
min         3.000000       2.000000       0.000000   
25%        48.000000     656.000000   14094.250000   
50%        52.000000     656.000000   32115.500000   
75%        65.0000

In [205]:
print(df["CompanyName"].nunique(), "unique companies")
print(df["ProductName"].nunique(), "unique products")
print(df["BrandName"].nunique(), "unique brands")
print(df["PrimaryCategory"].nunique(), "unique primary categories")
print(df["SubCategory"].nunique(), "unique subcategories")
print(df["ChemicalName"].nunique(), "unique chemicals")


604 unique companies
33438 unique products
2393 unique brands
13 unique primary categories
89 unique subcategories
123 unique chemicals
